In [1]:
### Benjamin Tollison ###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from decimal import Decimal
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

### Wing Constants

In [2]:
wing_area = 400
root_cord = 17 #ft
aspect_ratio = 3.6
quarter_cord_sweep = 45*deg2rad
low_speed_lift_curve_slope = 5.6
x_apex_of_wing = 16 #ft
incidence_angle = 0
# AC_wing = 32*mean_cord_length
mach = 0.56

# Task 1
Determine the trim AoA, tail incidence angle, equilibrium thrust, and the normal force with the following system of equations in the body-axis system:
$$\sum{F_X} = 0 = T_0 - W\sin\alpha_0 + L_{WF}\sin\alpha_0-D_0\cos\alpha_0 + L_H\sin(\alpha_0-\epsilon)$$
$$\sum{F_Z}=0 = -F_{N_0} + W\cos\alpha_0 - L_{WF}\cos\alpha_0-D_0\sin\alpha_0 - L_H\cos(\alpha_0-\epsilon)$$
$$\sum{M_{cg}}= 0 = F_{N_0}(X_{cg}-X_{inlet}) - L_{WF}(X_{AC_{WF}}-X_{cg})\cos\alpha_0 - D_0(X_{AC_{WF}}-X_{cg})\sin\alpha_0 - L_{H}(X_{AC_{H}}-X_{cg})\cos(\alpha_0-\epsilon)$$
## Finding $L_{WF}$
-Neglecting the lift from the fuselage
$$L_{WF} = L_W = q_\infty S_W C_{L_W}$$
- Find $C_{L_W} \approx C_{L_{W0}} + C_{L_{\alpha,W}} \alpha_0|C_{L_{W0}}=0,{symmetry}$
### Finding $C_{L_{\alpha,W}}$
- Find Taper ratio $\lambda = \frac{2S_w}{bc_r}$
- Find SweepAngle of the LE: $\Lambda_{LE} = \tan^{-1}\left(\tan{\Lambda_{c/4}} + \frac{1}{AR}\frac{1-\lambda}{1+\lambda}\right)$
- Find SweepAngle of the c/2: $\Lambda_{c/2} = \tan^{-1}\left(\tan{\Lambda_{LE}} - \frac{2}{AR}\frac{1-\lambda}{1+\lambda}\right)$
- Find $C_{l_\alpha} = \frac{2\pi}{\sqrt{1-M_\infty^2}}$
- Find Beta/k constants: $\beta = \sqrt{1-M^2}$, $k = \frac{C_{l_\alpha}}{2\pi}$
- plug into equation:
$$C_{L_\alpha} = \frac{2\pi AR}{2+\sqrt{\frac{AR^2\beta^2}{k^2}\left(1+\frac{\tan^2{\Lambda_{c/2}}}{\beta^2}\right)+4}} $$
### Find $q_\infty$
$$q_\infty = \frac{1}{2} \rho_\infty v_\infty^2$$
- Get speed of sound from the tables: $a = 296.5338[m/s]$
- Find $v_\infty$ from $M = \frac{v_\infty}{a}$
- Get the $\rho_\infty$ with isentropic equation
  - $\frac{\rho_0}{\rho} = \left(1+\frac{\gamma-1}{2}M^2\right)^\frac{1}{\gamma-1}$
  - $\rho_0 = 7.3654\cdot10^{-4} [sl/ft^3] = 0.3795971165326[kg/m^3]$

## $$\therefore L_{WF} = q_\infty S_W C_{L_{\alpha,W}} \alpha_0$$

In [3]:
wing_span = np.sqrt(wing_area*aspect_ratio)
wing_taper_ratio = (2*wing_area)/(wing_span*root_cord) - 1
sweepangle_LE = np.arctan(np.tan(quarter_cord_sweep)+(1/aspect_ratio)*((1-wing_taper_ratio)/(1+wing_taper_ratio)))
sweepangle_c2 = np.arctan(np.tan(sweepangle_LE)-(2/aspect_ratio)*((1-wing_taper_ratio)/(1+wing_taper_ratio)))
beta = np.sqrt(1-mach**2)
section_lift_coefficient = 5.6 # (2*np.pi)/np.sqrt(1-mach**2)
k = section_lift_coefficient/(2*np.pi)
CLalphaW = (2*np.pi*aspect_ratio)/(2+np.sqrt(((aspect_ratio**2*beta**2)/(k**2))*(1+(np.tan(sweepangle_c2)**2)/beta**2)+4))
displayVariable('C_{L_{\\alpha\\,W}}',CLalphaW)
rho_stagnation = 0.3795971165326
rho_infty = rho_stagnation*(1+.2*mach**2)**(1/(-.4))
speed_of_sound = 296.5338
velocity_of_freestream = mach*speed_of_sound
dynamic_pressure_freestream = 0.5*rho_infty*velocity_of_freestream**2
displayVariableWithUnits('q_\\infty',dynamic_pressure_freestream,'\\frac{kg}{m\\cdot{s^2}}')
q_infty = 0.5*0.00073654*(1+.2*mach**2)**(1/(-.4))*(mach*972.88)**2
displayVariableWithUnits('q_\\infty',q_infty,'\\frac{sl}{ft\\cdot{s^2}}')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Finding $D_0$
$$ \rightarrow D_0 \equiv D(\alpha_0)$$
$$ D = q_\infty S_W C_D$$
- Find $C_D$ from the given equation: $ C_D = 0.0145 + 0.1 C_{L_W}^2$
  - with $C_{L_W} = C_{L_{\alpha,W}}\alpha_0$
### $$\therefore D_0 = q_\infty S_W (0.0145 + 0.1C_{L_{\alpha,W}}^2 \alpha_0^2)$$

## Finding $L_H$
$$ L_H = \eta_H \frac{S_H}{S_W} C_{L_H}$$
### Finding $C_{L_H}$
$ C_{L_H} \equiv C_{L_H}(\alpha,\epsilon,i_H)$
- $\epsilon$ is also a function of $\alpha$
- $C_{L_H} \approx C_{L_{H0}} + C_{L_{\alpha,H}}\alpha_H + C_{L_{i_H}} i_H | C_{L_{H0}} = 0,{symmetry}$
- set $\alpha_H = \alpha_0 + i_H - \frac{d\epsilon}{d\alpha_W}\alpha_0$
- Find $C_{L_\alpha,H}$ with the same process as the wing
  - Find Taper ratio $\lambda = \frac{2S_w}{bc_r}$
  - Find SweepAngle of the LE: $\Lambda_{LE} = \tan^{-1}\left(\tan{\Lambda_{c/4}} + \frac{1}{AR}\frac{1-\lambda}{1+\lambda}\right)$
  - Find SweepAngle of the c/2: $\Lambda_{c/2} = \tan^{-1}\left(\tan{\Lambda_{LE}} - \frac{2}{AR}\frac{1-\lambda}{1+\lambda}\right)$
  - Find $C_{l_\alpha} = \frac{2\pi}{\sqrt{1-M_\infty^2}}$
  - Find Beta/k constants: $\beta = \sqrt{1-M^2}$, $k = \frac{C_{l_\alpha}}{2\pi}$
  - plug into equation:
  $C_{L_{\alpha,H}} = \frac{2\pi AR}{2+\sqrt{\frac{AR^2\beta^2}{k^2}\left(1+\frac{\tan^2{\Lambda_{c/2}}}{\beta^2}\right)+4}} $
- Find $C_{L_{i_H}}$ from the given equation: $C_{L_{i_H}} = C_{L_{\alpha,H}}\eta_H\frac{S_H}{S_W}$
### $$ \therefore L_H = \eta_H \frac{S_H}{S_W} C_{L_{\alpha,H}}\left(\alpha_0 + i_H - \frac{d\epsilon}{d\alpha_W}\alpha_0\right) + C_{L_{\alpha,H}}\eta_H\frac{S_H}{S_W} i_H $$ 

### Horizontal tail constants

In [4]:
tail_aspect_ratio = 3.4
tail_area = 100
quarter_cord_sweep = 45*deg2rad
tail_root_cord = 10
tail_efficieny_factor = 0.5
downwash_gradiant = 0.75
tail_span = np.sqrt(tail_area*tail_aspect_ratio)
tail_taper_ratio = (2*tail_area)/(tail_span*tail_root_cord) - 1
sweepangle_LE = np.arctan(np.tan(quarter_cord_sweep)+(1/tail_aspect_ratio)*((1-tail_taper_ratio)/(1+tail_taper_ratio)))
sweepangle_c2 = np.arctan(np.tan(sweepangle_LE)-(2/tail_aspect_ratio)*((1-tail_taper_ratio)/(1+tail_taper_ratio)))
beta = np.sqrt(1-mach**2)
section_lift_coefficient = 6 # (2*np.pi)/np.sqrt(1-mach**2)
k = section_lift_coefficient/(2*np.pi)
CLalphaH = (2*np.pi*tail_aspect_ratio)/(2+np.sqrt(((tail_aspect_ratio**2*beta**2)/(k**2))*(1+(np.tan(sweepangle_c2)**2)/beta**2)+4))
displayVariable('C_{L_{\\alpha\\,H}}',CLalphaH)
CLiH = CLalphaH*tail_efficieny_factor*(tail_area/wing_area)
displayVariable('C_{L_{i_H}}',CLiH)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Finding $F_{N_0}$
$$F_{N_0} = 2 q_\infty A_{inlet} \cos\alpha_0^2 \sin\alpha_0$$
- $A_{inlet}$ is given as 15 $ft^2$ or 1.39355 $m^2$
### Finding $\epsilon$
- $\epsilon$ estimated purely as a function of $\alpha$
$$\epsilon = \frac{d\epsilon}{d\alpha_W} (\alpha_W-\alpha_0)$$
where $\alpha_0$ is 0 because it is a symmetrical airfoil
### $$ \therefore \epsilon  = \frac{d\epsilon}{d\alpha}\alpha_0 $$

## Last is to find the $X_{CG},X_{AC_{WF}},X_{inlet},X_{AC_H}$
- $X_{CG}$ is given at 27 ft or 8.2296 m
- $X_{AC_{WF}} = X_{LE}+X_{apex,{WF}} + .32\bar{c}_W$
  - $X_{apex,WF}$ = 16 ft or 4.8768 m
  - $X_{LE} = \frac{b}{2}\left[\frac{1+2\lambda}{3(1+\lambda)}\right]c_r$
- $X_{AC_{H}} = X_{LE}+X_{apex,{H}} + .32\bar{c}_H$
  - $X_{apex,H}$ = 36 ft or 10.9728 m 
- Where $\bar{c}_\circ = \frac{2\left(1+\lambda+\lambda^2\right)}{3\left(1+\lambda\right)}c_{r\circ}$\
$X_{inlet}$ is at the nose so it is equal to 0

In [5]:
mean_cord_length = lambda taper_ratio,rootcord: ((2*(1+taper_ratio+taper_ratio**2))/(3*(1+taper_ratio)))*rootcord
x_cg = 27 # 8.2296
x_apex_wf = 16# 4.8768
x_apex_h = 36
x_LE = 8.83299
x_ac_wf = x_LE + x_apex_wf + .32*mean_cord_length(wing_taper_ratio,root_cord)
x_ac_h = x_LE + x_apex_h + .32*mean_cord_length(tail_taper_ratio,tail_root_cord)
x_inlet = 0
displayVariableWithUnits('X_{CG}',x_cg,'ft')
displayVariableWithUnits('X_{inlet}',x_inlet,'ft')
displayVariableWithUnits('X_{AC_{WF}}',x_ac_wf,'ft')
displayVariableWithUnits('X_{AC_{H}}',x_ac_h,'ft')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Now to solve the system of equations

In [6]:
def SystemOfEqs(input_array):
  T0,alpha0,ih = input_array
  W = 22000
  area_inlet = 15
  epsilon = 0.75*alpha0
  Lwf = q_infty*wing_area*CLalphaW*alpha0
  D0 = q_infty*wing_area*(0.0145+0.1*(CLalphaW*alpha0)**2)
  Fn0 = 2*q_infty*area_inlet*np.cos(alpha0)**2*np.sin(alpha0)
  Lh = tail_efficieny_factor*(tail_area/wing_area)*CLalphaH*(alpha0+ih-epsilon)+CLalphaH*tail_efficieny_factor*(tail_area/wing_area)*ih
  x_direction = T0 - W*np.sin(alpha0) + Lwf*np.sin(alpha0)-D0*np.cos(alpha0)+Lh*np.sin(alpha0-epsilon)
  z_direction = W*np.cos(alpha0) - Fn0 - Lwf*np.cos(alpha0) - D0*np.sin(alpha0)-Lh*np.cos(alpha0-epsilon)
  y_moment = Fn0*abs(x_cg-x_inlet)-Lwf*abs(x_ac_wf-x_cg)*np.cos(alpha0) - D0*abs(x_ac_wf-x_cg)*np.sin(alpha0) - Lh*abs(x_ac_h-x_cg)*np.cos(alpha0-epsilon)
  return [x_direction,z_direction,y_moment]
T0,alpha0,ih = scipy.optimize.fsolve(SystemOfEqs,[2000,np.radians(10),np.radians(-5)])
ih = -((np.arcsin(np.sin(ih))*rad2deg)%20)
displayVariable('T_0',T0)
displayVariable('\\alpha_0',alpha0*rad2deg)
displayVariable('i_H',ih)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>